In [ ]:
import folium

In [ ]:
import json
import pandas as pd
import re
import geopandas as gpd

## Property location

In [ ]:
# we want to draw a map to show all the locations of the extracted properties
m = folium.Map([-38.013995, 145.264296], zoom_start=9)

df = pd.read_csv('../data/curated/property.csv')

for index, row in df.iterrows():
    lat = float(re.findall(r'\[(-\d+.\d+)',row['coordinates'])[0])
    long = float(re.findall(r'(\d+.\d+)\]',row['coordinates'])[0])
    folium.Marker([lat, long]).add_to(m)

m.save('../plots/property.html')
m

## Property location with defined boundary of SA2

In [ ]:
# we add boundary of each Statistical Area 2 into the property map
gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)
# create a JSON 
geoJSON = gdf[['SA2_NAME21', 'geometry']].to_json()

m.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))
m.save('../plots/property_sa2.html')
m

## 2021 Population Density

In [98]:
import csv

In [102]:
# we draw a map to represent the density of population in each SA2
tmp_lst = []
with open('../data/curated/population.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df_popu = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])

gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)
gdf.rename(columns={'SA2_CODE21':'SA2 code'}, inplace=True)

merge_df = pd.merge(df_popu,gdf,on='SA2 code')
merge_df = merge_df[['SA2 code', 'SA2 name', 'Population density 2021','geometry']]

m = folium.Map([-38.013995, 145.264296], zoom_start=9)

m.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))





      SA2 code             SA2 name Population density 2021  \
0    201011001            Alfredton                   319.2   
1    201011002             Ballarat                   975.5   
2    201011005            Buninyong                   140.2   
3    201011006            Delacombe                   311.5   
4    201011007        Smythes Creek                    40.2   
..         ...                  ...                     ...   
517  217031476                Otway                     2.6   
518  217041477         Moyne - East                     2.1   
519  217041478         Moyne - West                     4.9   
520  217041479  Warrnambool - North                   359.9   
521  217041480  Warrnambool - South                   110.6   

                                              geometry  
0    POLYGON ((143.78281 -37.56667, 143.75557 -37.5...  
1    POLYGON ((143.81896 -37.55583, 143.81644 -37.5...  
2    POLYGON ((143.84170 -37.61597, 143.84175 -37.6...  
3    POLYGON ((

## School location combined with property location